## In this notebook, we are going to consolidate the common computer vision algorithms that are often use in the object detection pipeline
1. Non Max Supression

## Non Max supression
NMS is a common strategy to reduce the number of bounding box.
### References
1. https://www.youtube.com/watch?v=VAo84c1hQX8

### Steps 
1. Filter out all the low probability boxes
2. Sort the bbox from highest confidence to lowest confidence
3. From the highest confidence box, remove other boxes of the same class with high IOU

## Torch

In [ ]:
import torch

In [ ]:
def IOU_torch(box_preds, box_labels, format="xyxy"):
    '''
    Calculates IOU for bboxes
    '''
    if format == "xyxy":
        bbox1_x1 = box_preds[..., 0:1] # shape = (n, 1)
        bbox1_y1 = box_preds[..., 1:2]
        bbox1_x2 = box_preds[..., 2:3]
        bbox1_y2 = box_preds[..., 3:4]
        bbox2_x1 = box_labels[..., 0:1] 
        bbox2_y1 = box_labels[..., 1:2]
        bbox2_x2 = box_labels[..., 2:3]
        bbox2_y2 = box_labels[..., 3:4]

    elif format == "xywh": #yoloV5 format
        bbox1_x1 = box_preds[..., 0:1] - box_preds[..., 2:3] / 2
        bbox1_y1 = box_preds[..., 1:2] - box_preds[..., 3:4] / 2
        bbox1_x2 = box_preds[..., 0:1] + box_preds[..., 2:3] / 2
        bbox1_y2 = box_preds[..., 1:2] + box_preds[..., 3:4] / 2

        bbox2_x1 = box_labels[..., 0:1] - box_labels[..., 2:3] / 2
        bbox2_y1 = box_labels[..., 1:2] - box_labels[..., 3:4] / 2
        bbox2_x2 = box_labels[..., 0:1] + box_labels[..., 2:3] / 2
        bbox2_y2 = box_labels[..., 1:2] + box_labels[..., 3:4] / 2

    else:
        raise NotImplementedError("This bbox format is not supported")

    bbox1_area =  (bbox1_x2-bbox1_x1) * (bbox1_y2-bbox1_y1)
    bbox2_area =  (bbox2_x2-bbox2_x1) * (bbox2_y2-bbox2_y1)
    
    intersection_x1 = torch.max(bbox1_x1,bbox2_x1)
    intersection_y1 = torch.max(bbox1_y1,bbox2_y1)
    intersection_x2 = torch.min(bbox1_x2,bbox2_x2)
    intersection_y2 = torch.min(bbox1_y2,bbox2_y2)

    # if the box dont intersect, one or more of the difference will be negative
    intersection_area = torch.clamp(intersection_x2-intersection_x1, min=0) * torch.clamp(intersection_y2-intersection_y1, min=0) 

    return intersection_area / (bbox1_area + bbox2_area- intersection_area + 1e-7) #epilson for stability

In [ ]:
def non_max_supression(predictions, iou_threshold, confidence_threshold, box_format='xyxy'):
    """
    prediction structure assumption = [[class, proba, x1,y1,x2,y2] ...]
    """

    high_confidence_predictions = [pred for pred in predictions if pred[1]>=confidence_threshold]
    sorted_predictions = sorted(high_confidence_predictions, key=lambda x:x[1], reverse=True)
    bbox_after_nms = []

    while sorted_predictions:
        chosen_box = sorted_predictions.pop(0)
        bbox_after_nms.append(chosen_box)

        sorted_predictions = [pred for pred in sorted_predictions if ((pred[0]!=chosen_box[0]) or (IOU_torch(torch.tensor(pred[-4:]), torch.tensor(chosen_box[-4:]), format=box_format)<iou_threshold))]

    return bbox_after_nms



In [ ]:
t1_boxes = [
            [1, 1, 0.5, 0.45, 0.4, 0.5],
            [1, 0.8, 0.5, 0.5, 0.2, 0.4],
            [1, 0.7, 0.25, 0.35, 0.3, 0.1],
            [1, 0.05, 0.1, 0.1, 0.1, 0.1],
        ]

assert non_max_supression(t1_boxes,iou_threshold=0.2,confidence_threshold=7 / 20, box_format='xywh') == [[1, 1, 0.5, 0.45, 0.4, 0.5], [1, 0.7, 0.25, 0.35, 0.3, 0.1]], "test_1 failed"